In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import requests
import json
import time
import re
from pyresparser import ResumeParser
import PyPDF2
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from bayes_opt import BayesianOptimization
import warnings
warnings.filterwarnings('ignore')
# import scattertext as st
# from pandas import json_normalize
# from nltk.stem import WordNetLemmatizer
# import spacy
# import texthero as hero
# from wordcloud import WordCloud
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process

%matplotlib inline

In [ ]:
# df['jd_text'] = df['Job Description'].pipe(hero.clean)
# text = " ".join(df['jd_text'].values)

# def make_wordcloud(text):

#     wordcloud = WordCloud(
#         width=800,
#         height=800,
#         min_font_size=10,
#         background_color="black",
#         colormap="Set2",
#         collocation_threshold=3,
#     ).generate(text)

#     fig = plt.figure(figsize=(8, 8), facecolor=None)
#     plt.imshow(wordcloud, interpolation="bilinear")
#     plt.axis("off")
#     plt.tight_layout(pad=0)
#     plt.show()
#     return fig

# text_cloud = make_wordcloud(text)

In [ ]:
# Tokenize text
# filtered_df = df.copy()
# filtered_df["parse"] = filtered_df['Job Description'].apply(st.whitespace_nlp_with_sentences)

In [ ]:
# corpus = (
#     st.CorpusFromParsedDocuments(filtered_df, category_col="job_title_category", parsed_col="parse")
#     .build()
#     .get_unigram_corpus()
#     .compact(st.AssociationCompactor(2000))
# )

In [ ]:
# used for tagging words with their parts of speech
# nltk.download("averaged_perceptron_tagger")

# term_freq_df = corpus.get_term_freq_df()
# term_freq_df["data_scientist_score"] = corpus.get_scaled_f_scores("data_scientist")
# term_freq_df["data_analyst_score"] = corpus.get_scaled_f_scores("data_analyst")
# term_freq_df["data_engineer_score"] = corpus.get_scaled_f_scores("data_engineer")
# term_freq_df["ml_engineer_score"] = corpus.get_scaled_f_scores("ml_engineer")
# term_freq_df["others_score"] = corpus.get_scaled_f_scores("others")

# # Remove terms that are not nouns
# def is_noun(word: str):
#     pos = nltk.pos_tag([word])[0][1]
#     return pos[:2] == "NN"

# term_freq_df = term_freq_df.loc[map(is_noun, term_freq_df.index)]

In [ ]:
# term_freq_df.sort_values(by='data_scientist_score', ascending=False).index[:30]
# term_freq_df.sort_values(by='data_analyst_score', ascending=False).index[:30]
# term_freq_df.sort_values(by='data_engineer_score', ascending=False).index[:30]
# term_freq_df.sort_values(by='ml_engineer_score', ascending=False).index[:30]

In [ ]:
# url = "https://auth.emsicloud.com/connect/token"
# CLIENT_ID = 'guntc5o6ib9mqb8x'
# CLIENT_SECRET = '0ItWAUes'

# payload = "client_id=" + CLIENT_ID + "&client_secret=" + CLIENT_SECRET + "&grant_type=client_credentials&scope=emsi_open"
# headers = {'Content-Type': 'application/x-www-form-urlencoded'}
# response = requests.request("POST", url, data=payload, headers=headers)
# access_token = json.loads(response.text)['access_token']
# print(access_token)

In [ ]:
# url = "https://emsiservices.com/skills/versions/latest/skills"
# auth = 'Bearer ' + access_token 
# headers = {'Authorization': auth}
# querystring = {"fields": "name, type"}
# response = requests.request("GET", url, headers=headers, params=querystring)
# response = response.json()['data']
# all_skills_df = pd.DataFrame(json_normalize(response))

In [ ]:
# all_skills_df = all_skills_df[all_skills_df['type.name'] == 'Hard Skill']
# all_skills_df.head()

In [ ]:
# all_skills_df = pd.read_csv('all_skills_df.csv')

In [ ]:
# all_skills_df['name'] = all_skills_df['name'].str.lower()
# all_skills_df['name'] = all_skills_df['name'].apply(lambda x: x.split())
# all_skills_df = all_skills_df.explode('name')
# all_skills = set(all_skills_df['name'])
# print(all_skills)

In [ ]:
# skills = []
# for word in text:
#     if word in all_skills and len(word) > 1 and not word.isdigit():
#         skills.append(word)
# print(skills)

In [ ]:
# en = spacy.load('en_core_web_sm')
# sw_spacy = en.Defaults.stop_words

In [ ]:
# new_skill_list = [word for word in skills if word not in sw_spacy]
# print(new_skill_list)

In [ ]:
# skill_set = set()
# nltk.download('wordnet')
# lemmatizer = WordNetLemmatizer()

# for word in new_skill_list:
#     skill_set.add(lemmatizer.lemmatize(word))

In [ ]:
# text_2 = " ".join(text)
# text_file = open('text.txt', 'w')
# _ = text_file.write(text_2)
# text_file.close()

In [ ]:
# from resume_parser import resumeparse
# data_2 = resumeparse.read_file('check.docx')

In [ ]:
# from pyresparser import ResumeParser
# data = ResumeParser('check.docx').get_extracted_data()

In [ ]:
df = pd.read_csv('data.csv')
df.drop(['index'], axis=1, inplace=True)
df.head()

In [ ]:
df['job_title_category'] = df['Job Title'].apply(lambda x: 
    'data_engineer' if 'intelligence anal' in x.lower()
    else 'data_engineer' if 'data engineer' in x.lower()
    else 'ml_engineer' if 'machine learning' in x.lower()
    else 'data_scientist' if 'data scien' in x.lower()
    else 'data_analyst' if 'anal' in x.lower()
    else 'unknown')
job_titles = list(df['job_title_category'].unique())
print(job_titles)

In [ ]:
df['Degree'] = df['Degree'].replace({'na': 'B'})
df['Degree'] = df['Degree'].apply(lambda x: ['B', 'M', 'P'] if x == 'P' else ['B', 'M'] if x == 'M' else ['B'])

In [ ]:
SKILLS_DB = ['data-science', 'machine-learning', 'cloud', 'database-management', 'bigdata',
             'data-analytics', 'business-intelligence', 'analytics', 'statistics', 'data-visualization']
skills_stack_overflow = []
for skills in SKILLS_DB:
    url = "https://api.stackexchange.com/2.3/tags/" + skills + "/related?site=stackoverflow"
    response = requests.request("GET", url)
    response = json.loads(response.text)
    time.sleep(3)
    for i in range(len(response['items'])):
        skills_stack_overflow.append(response['items'][i]['name'])

In [ ]:
SKILLS_DB.extend(['aws', 'gcp', 'spark', 'bi', 'tableau', 'clustering', 'recommendation', 
                  'rnn', 'recurringneuralnetwork', 'cnn', 'redshift', 'kafka', 'redis', 'sas', 
                  'bayesian', 'bayesiananalysis'])

In [ ]:
skills_stack_overflow.extend(SKILLS_DB)
skills_stack_overflow = list(set(skills_stack_overflow))
skills_stack_overflow = [x.replace('-', '') for x in skills_stack_overflow]
skills_stack_overflow = set(skills_stack_overflow)
skills_stack_overflow = skills_stack_overflow - set(['charts', 'visualization', 'plot', 'csv', 'ios', 'logging', 
                                                    'random', 'businessobjects', 'facebook', 'android', 'dataset', 
                                                    'report', 'server', 'parseplatform', 'tracking', 'performance'])
print(sorted(list(skills_stack_overflow)))

In [ ]:
skills = set()
jd = df['Job Description']
df['skills'] = np.empty((len(df), 0)).tolist()
char_to_int = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9, 'ten': 10, 
              'eleven': 11, 'twelve': 12, 'thirteen': 13, 'fourteen': 14, 'fifteen': 15, 'sixteen': 16}
df['experience'] = 'Unknown'
for i in range(len(jd)):
    temp = set()
    text = jd[i]
    text = text.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    text = tokenizer.tokenize(text)
    for word in text:
        if word in skills_stack_overflow and word not in temp:
                skills.add(word)
                temp.add(word)
    bigrams = list(map(' '.join, nltk.everygrams(text, 2, 2)))
    bigrams = [x.replace(' ', '') for x in bigrams]
    for bigram in bigrams:
        if bigram in skills_stack_overflow and bigram not in temp:
            skills.add(bigram)
            temp.add(bigram)
    if len(temp) < 5:
        df['skills'][i] = ['Unknown']
    else:
        df['skills'][i].extend(temp)
    text = " ".join(text)
    text_copy = text
    start_idx = text.find('qualification')
    if start_idx == -1:
        start_idx = text.find('requirements')
    if start_idx != -1:
        text = text[start_idx:]
    end_idx = text.find('experience') + 40
    if start_idx == -1 and end_idx == 39:
        pass
    else: 
        text = text[:end_idx + 1]
    rx = re.compile('\w+(?=\s+year)')
    is_found = rx.search(text)
    if is_found:
        match = is_found.group(0)
        if match in char_to_int:
            exp = char_to_int[match]
            if exp <= 2:
                exp = 'Entry'
            elif exp < 5:
                exp = 'Mid'
            elif exp <= 16:
                exp = 'Senior'
            df['experience'][i] = exp
        elif match.isdigit():
            exp = int(match)
            if exp <= 2:
                exp = 'Entry'
            elif exp < 5:
                exp = 'Mid'
            elif exp <= 15:
                exp = 'Senior'
            if type(exp) == str:
                df['experience'][i] = exp
    else:
        experience_types = ['years of experience', 'years experience', 'years relevant experience', 
                           'years professional experience']
        flag = False
        for experience_type in experience_types:
            rx = re.compile('\w+(?=\s+' + experience_type + ')')
            is_found = rx.search(text_copy)
            if is_found:
                match = is_found.group(0)
                if match in char_to_int:
                    exp = char_to_int[match]
                    if exp <= 2:
                        exp = 'Entry'
                    elif exp <= 5:
                        exp = 'Mid'
                    elif exp <= 15:
                        exp = 'Senior'
                    df['experience'][i] = exp
                    flag = True
                    break
                elif match.isdigit():
                    exp = int(match)
                    if exp <= 2:
                        exp = 'Entry'
                    elif exp <= 5:
                        exp = 'Mid'
                    elif exp <= 15:
                        exp = 'Senior'
                    if type(exp) == str:
                        df['experience'][i] = exp
                        flag = True
                        break
        if not flag:
            job_title = df['Job Title'][i].lower()
            if 'manage' in job_title or 'direct' in job_title or 'principal' in job_title or 'lead' in job_title or 'architect' in job_title or 'project' in job_title or 'staff' in job_title:
                exp = 'Senior'
                df['experience'][i] = exp
            elif 'associate' in job_title or 'sr' in job_title or 'senior' in job_title or 'consult' in job_title or ' ii' or job_title:
                exp = 'Mid'
                df['experience'][i] = exp
print(skills)

In [ ]:
print(df.shape)
df['skill_len'] = df['skills'].apply(lambda x: len(x))
df = df[df['skill_len'] != 1]
salary_df = df.copy()
salary_df.to_csv('salary_df.csv', index=False)
df = df.reset_index()
print(df.shape)

In [ ]:
sectors = list(df['Sector'].unique())
sector_og = sectors.copy()
sectors.extend(['marketing', 'fashion', 'civil', 'construction', 'entertainment', 'art', 'oil', 'energy', 'supplychain', 
               'educationdepartment', 'educationsector', 'educationindustry'])
sectors = [x.lower() for x in sectors]
sectors = [x.split(' & ') for x in sectors]
sectors = [item for sublist in sectors for item in sublist]
sectors = [x.replace(',', '') for x in sectors]
sectors = [x.replace(' ', '') for x in sectors]
sectors.remove('utilities')
sectors.remove('artsentertainment')
sectors.remove('oilgasenergy')
sectors.remove('informationtechnology')
sectors.remove('education')
sectors.remove('-1')
sectors = [re.sub('s$', '', x) for x in sectors]
sectors_dict = {}
for sector in sectors:
    if sector == 'aerospace':
        sectors_dict[sector] = 'defence'
    elif sector == 'oil':
        sectors_dict[sector] = 'energy'
    elif sector == 'finance' or sector == 'insurance' or sector == 'accounting':
        sectors_dict[sector] = 'banking'
    elif sector == 'biotech' or sector == 'pharmaceutical':
        sectors_dict[sector] = 'healthcare'
    elif sector == 'media':
        sectors_dict[sector] = 'marketing'
    elif sector == 'logistic' or sector == 'supplychain':
        sectors_dict['logistics'] = 'transportation'
    elif sector == 'metal':
        sectors_dict[sector] = 'mining'
    elif sector == 'tourism':
        sectors_dict[sector] = 'travel'
    elif sector == 'non-profit':
        sectors_dict[sector] = 'government'
    elif sector == 'recreation' or sector == 'fashion' or sector == 'art':
        sectors_dict[sector] = 'entertainment'
    elif sector == 'construction':
        sectors_dict[sector] = 'civil'
    elif sector == 'educationdepartment' or sector == 'educationsector' or sector == 'educationindustry':
        sectors_dict[sector] = 'education'
    elif sector == 'forestry':
        sectors_dict[sector] = 'agriculture'
    else:
        sectors_dict[sector] = sector
print(sectors_dict)

In [ ]:
df['sector_category'] = df['Sector'].str.lower()
df['sector_category'] = df['sector_category'].apply(lambda x: x.replace(' & ', ''))
df['sector_category'] = df['sector_category'].apply(lambda x: x.replace(',', ''))
df['sector_category'] = df['sector_category'].apply(lambda x: x.replace(' ', ''))
df['sector_category'] = df['sector_category'].apply(lambda x:
    'defence' if 'defense' in x
    else 'energy' if 'energy' in x 
    else 'banking' if 'finance' in x
    else 'healthcare' if 'pharma' in x
    else 'marketing' if 'media' in x 
    else 'banking' if 'insurance' in x
    else 'transportation' if 'transportation' in x
    else 'telecommunication' if 'telecommunications' in x
    else 'mining' if 'mining' in x
    else 'agriculture' if 'agriculture' in x
    else 'travel' if 'travel' in x
    else 'government' if 'non-profit' in x
    else 'entertainment' if 'entertainment' in x
    else 'civil' if 'construction' in x
    else 'banking' if 'accounting' in x
    else 'unknown' if '-1' in x
    else x)

In [ ]:
df_copy = df.copy()
df['index'] = df.index
df = df[['index', 'Job Title', 'company_txt', 'Location', 'job_title_category', 'experience', 'sector_category', 'Degree', 'skills', 'Size']]
df.to_csv('stage.csv', index=False)
print(df['index'].max())

In [ ]:
skills_df = df[['index', 'experience', 'skills']]
previous_job_df = df[['index', 'experience', 'job_title_category']]
industry_df = df[['index', 'experience', 'sector_category']]
degree_df = df[['index', 'experience', 'Degree']]

In [ ]:
skills_df = skills_df.explode('skills')
degree_df = degree_df.explode('Degree')

In [ ]:
skills_df = pd.get_dummies(skills_df, columns=['skills'], prefix='col_')
skills_df = skills_df.groupby(['index', 'experience']).sum().reset_index()
skills_df.to_csv('skills_df.csv', index=False)
print(skills_df.shape)

In [ ]:
degree_df = pd.get_dummies(degree_df, columns=['Degree'], prefix='col_')
degree_df = degree_df.groupby(['index', 'experience']).sum().reset_index()
degree_df.to_csv('degree_df.csv', index=False)
print(degree_df.shape)

In [ ]:
previous_job_df = pd.get_dummies(previous_job_df, columns=['job_title_category'], prefix='col_')
previous_job_df = previous_job_df.groupby(['index', 'experience']).sum().reset_index()
previous_job_df.to_csv('previous_job_df.csv', index=False)
print(previous_job_df.shape)

In [ ]:
industry_df = pd.get_dummies(industry_df, columns=['sector_category'], prefix='col_')
industry_df = industry_df.groupby(['index', 'experience']).sum().reset_index()
industry_df.to_csv('industry_df.csv', index=False)
print(industry_df.shape)

In [ ]:
salary_df = salary_df.drop(['Job Title', 'Salary Estimate', 'Job Description', 'Company Name', 'Headquarters', 
                            'Type of ownership', 'Industry', 'Sector', 'Competitors', 'Hourly','Employer provided', 
                            'Lower Salary', 'Upper Salary', 'company_txt', 'Job Location', 
                            'Founded', 'Rating', 'Age', 'Revenue', 'skill_len'], axis=1).reset_index()
temp = salary_df.copy()
salary_df.head()

In [ ]:
salary_df['Size'].value_counts()
salary_df = salary_df[salary_df['Size'] != 'unknown']

In [ ]:
salary_df['Size'] = salary_df['Size'].apply(lambda x: 
                                    1 if 'Jan-50' in x else 1 if '51 - 200' in x
                                    else 2 if '201 - 500' in x else 3 if '501 - 1000' in x
                                    else 4 if '1001 - 5000' in x else 5 if '5001 - 10000' in x else 6)
salary_df['Size'].value_counts()

In [ ]:
salary_df['city'] = salary_df['Location'].apply(lambda x: x.split(',')[0])
salary_df['state'] = salary_df['Location'].apply(lambda x: x.split(',')[1])
salary_df.drop(['Location'], axis=1, inplace=True)
salary_df.head()

In [ ]:
salary_df['salary'] = 0.87 * salary_df['Avg Salary(K)']
salary_df.drop(['Avg Salary(K)'], axis=1, inplace=True)
salary_df.head()

In [ ]:
cost_of_living_df = pd.read_csv('cost_of_living.csv')
cost_of_living_df = cost_of_living_df[['city', 'state', 'cost_of_living']]
cost_of_living_df.head()

In [ ]:
salary_df = salary_df.merge(cost_of_living_df, how='left', on=['city', 'state'])
salary_df.head()

In [ ]:
salary_df.isna().sum()

In [ ]:
salary_df['cost_of_living'] = salary_df['cost_of_living'].fillna(salary_df.groupby(['state'])['cost_of_living'].transform('mean'))
mean_cost_of_living = np.mean(salary_df['cost_of_living'])
cost_of_living_state = salary_df[['state', 'cost_of_living']]
cost_of_living_state = cost_of_living_state.dropna()
cost_of_living_state = cost_of_living_state.groupby(['state']).agg(mean_val = ('cost_of_living', 'mean')).reset_index()
cost_of_living_state = cost_of_living_state.set_index('state').T.to_dict('records')[0]
cost_of_living_state['all'] = mean_cost_of_living
salary_df['cost_of_living'] = salary_df['cost_of_living'].fillna(np.mean(salary_df['cost_of_living']))
salary_df.isna().sum()

In [ ]:
salary_df = salary_df.drop(['city', 'state'], axis=1)
salary_df.head()

In [ ]:
skills_encode_df = salary_df[['index', 'skills']]
degree_encode_df = salary_df[['index', 'Degree']]
salary_df = salary_df.drop(['skills', 'Degree'], axis=1)
skills_encode_df = skills_encode_df.explode('skills')
degree_encode_df = degree_encode_df.explode('Degree')
skills_encode_df = pd.get_dummies(skills_encode_df, columns=['skills'], prefix='col_', drop_first=True)
degree_encode_df = pd.get_dummies(degree_encode_df, columns=['Degree'], prefix='col_', drop_first=True)
skills_encode_df = skills_encode_df.groupby(['index']).sum().reset_index()
degree_encode_df = degree_encode_df.groupby(['index']).sum().reset_index()
salary_df = salary_df.merge(skills_encode_df, how='inner', on='index')
salary_df = salary_df.merge(degree_encode_df, how='inner', on='index')
salary_df = pd.get_dummies(salary_df, columns=['job_title_category', 'experience'], prefix='col_', drop_first=True)
print(salary_df.shape)

In [ ]:
X = salary_df.drop(['salary'], axis=1)
Y = salary_df['salary']
X_skill = X[skills_encode_df.columns]
X_skill = X_skill.drop(['index'], axis=1)
X_skill = X_skill.to_numpy()
tsvd = TruncatedSVD(n_components=3, random_state=0)
W = tsvd.fit_transform(X_skill)
print(W.shape)
for i in range(W.shape[1]):
    col = 'skill_' + str(i)
    X[col] = W[:, i]
X = X.drop(skills_encode_df.columns, axis=1)
X.head()

In [ ]:
results_xgb = XGBRegressor(n_estimators=650, learning_rate=0.1, subsample=0.75, max_depth=7)
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
scores = cross_val_score(results_xgb, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = abs(scores)
print('Mean MAE and Std from CV: %.3f (%.3f)' % (scores.mean(), scores.std()))
results_xgb = XGBRegressor(n_estimators=650, learning_rate=0.1, subsample=0.75, max_depth=7).fit(X, Y)

In [ ]:
resume_data = ResumeParser('Aditya_Srivastava_Resume.pdf').get_extracted_data()
print(resume_data)

In [ ]:
experience = 'Unknown'
if resume_data['total_experience'] <= 2:
    experience = 'Entry'
elif resume_data['total_experience'] < 5:
    experience = 'Mid'
else:
    experience = 'Senior'
    
if not resume_data['degree']:
    education_level = ['B']
else:
    degrees = [x.lower() for x in resume_data['degree']]
    education_level = ['B']
    for degree in degrees:
        if 'phd' in degree:
            education_level = ['B', 'M', 'P']
            break
        elif 'postdoc' in degree:
            education_level = ['B', 'M', 'P']
            break
        elif 'master' in degree:
            education_level = ['B', 'M']
        elif 'masters' in degree:
            education_level = ['B', 'M']

In [ ]:
previous_job_title, skills, sectors = set(), set(), set()
pdfFileObj = open('Aditya_Srivastava_Resume.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pageObj = pdfReader.getPage(0)
corpus = pageObj.extractText()
pdfFileObj.close()
text = corpus.lower()
tokenizer = nltk.RegexpTokenizer(r"\w+")
text = tokenizer.tokenize(text)
for word in text:
        if word in skills_stack_overflow:
            skills.add(word)
        if word in sectors_dict:
            sectors.add(sectors_dict[word])
bigrams = list(map(' '.join, nltk.everygrams(text, 2, 2)))
bigrams = [x.replace(' ', '') for x in bigrams]
for bigram in bigrams:
    if bigram in skills_stack_overflow:
        skills.add(bigram)
    if bigram in sectors_dict:
        sectors.add(bigram)
    elif bigram == 'dataanalyst' or bigram == 'businessanalyst':
        previous_job_title.add('data_analyst')
    elif bigram == 'dataengineer' or bigram == 'businessintelligence':
        previous_job_title.add('data_engineer')
    elif bigram == 'datascientist':
        previous_job_title.add('data_scientist')
    elif bigram == 'learningengineer' or bigram == 'mlengineer':
        previous_job_title.add('ml_engineer')
if len(previous_job_title) == 0:
    previous_job_title.add('unknown')
if len(sectors) == 0:
    sectors.add('informationtechnology')

In [ ]:
print('Job titles:', previous_job_title)
print('Skills:', skills)
print('Sectors:', sectors)
print('Experience level:', experience)
print('Education level:', education_level)

In [ ]:
if experience != 'Unknown':
    skills_df = skills_df[skills_df['experience'] == experience]
    previous_job_df = previous_job_df[previous_job_df['experience'] == experience]
    industry_df = industry_df[industry_df['experience'] == experience]
    degree_df = degree_df[degree_df['experience'] == experience]
index_map = skills_df.index
print(skills_df.shape)
print(index_map)

In [ ]:
data = np.zeros(shape=(1, 3))
student_experience_df = pd.DataFrame(data, columns=['col__Mid', 'col__Senior', 'col__Unknown'])
if experience != 'Entry':
    experience_str = 'col__' + experience
    student_experience_df[experience_str] = 1.0
student_experience_df.head()

In [ ]:
data = np.zeros(shape=(1, len(skills_df.columns)))
student_skills_df = pd.DataFrame(data, columns=skills_df.columns)
student_skills_df = student_skills_df.drop(['index', 'experience'], axis=1)
skills_list = list(skills)
skills_list = ['col__' + skill for skill in skills_list]
student_skills_df[skills_list] = 1
student_skills_df.head()

In [ ]:
data = np.zeros(shape=(1, len(skills_df.columns)))
student_skills_df = pd.DataFrame(data, columns=skills_df.columns)
student_skills_df = student_skills_df.drop(['index', 'experience'], axis=1)
skills_list = list(skills)
skills_list = ['col__' + skill for skill in skills_list]
student_skills_df[skills_list] = 1
student_skills_df.head()

In [ ]:
data = np.zeros(shape=(1, len(previous_job_df.columns)))
student_previous_job_df = pd.DataFrame(data, columns=previous_job_df.columns)
student_previous_job_df = student_previous_job_df.drop(['index', 'experience'], axis=1)
previous_job_list = list(previous_job_title)
previous_job_list = ['col__' + previous_job for previous_job in previous_job_list]
student_previous_job_df[previous_job_list] = 1
student_previous_job_df.head()

In [ ]:
data = np.zeros(shape=(1, len(industry_df.columns)))
student_industry_df = pd.DataFrame(data, columns=industry_df.columns)
student_industry_df = student_industry_df.drop(['index', 'experience'], axis=1)
sectors_list = list(sectors)
sectors_list = ['col__' + sector for sector in sectors_list]
student_industry_df[sectors_list] = 1
student_industry_df.head()

In [ ]:
data = np.zeros(shape=(1, len(degree_df.columns)))
student_degree_df = pd.DataFrame(data, columns=degree_df.columns)
student_degree_df = student_degree_df.drop(['index', 'experience'], axis=1)
degree_list = list(education_level)
degree_list = ['col__' + degree for degree in degree_list]
student_degree_df[degree_list] = 1
student_degree_df.head()

In [ ]:
def dataframe_to_ndarray(df1, df2):
    a = df1.drop(['index', 'experience'], axis=1)
    a = a.to_numpy()
    b = df2.to_numpy()
    return a, b

def cosine_similarity(A,B):
    num = np.dot(A,B.T)
    p1 = np.sqrt(np.sum(A**2,axis=1))[:,np.newaxis]
    p2 = np.sqrt(np.sum(B**2,axis=1))[np.newaxis,:]
    return num / (p1 * p2)

In [ ]:
A, B = dataframe_to_ndarray(skills_df, student_skills_df)
skill_match_arr = cosine_similarity(A, B)
A, B = dataframe_to_ndarray(previous_job_df, student_previous_job_df)
title_match_arr = cosine_similarity(A, B)
A, B = dataframe_to_ndarray(industry_df, student_industry_df)
industry_match_arr = cosine_similarity(A, B)
A, B = dataframe_to_ndarray(degree_df, student_degree_df)
degree_match_arr = cosine_similarity(A, B)
print('Skills match:', np.argmax(skill_match_arr), np.max(skill_match_arr))
print('Title match:', np.argmax(title_match_arr), np.max(title_match_arr))
print('Industry match:', np.argmax(industry_match_arr), np.max(industry_match_arr))
print('Degree match:', np.argmax(degree_match_arr), np.max(degree_match_arr))

In [ ]:
results_arr = np.concatenate((skill_match_arr, title_match_arr, industry_match_arr, degree_match_arr), axis=1)
minmax_scaler = MinMaxScaler()
results_arr = minmax_scaler.fit_transform(results_arr)
weighted_arr = np.array([0.5, 0.2, 0.15, 0.15])[:, np.newaxis]
score_arr = np.matmul(results_arr, weighted_arr)
buffer = np.argsort(score_arr, axis=0)[-10:-1].flatten()
company_recommendations = {}
company_recommendations[df['company_txt'][index_map[np.argmax(score_arr)]]] = np.argmax(score_arr)
for i in range(len(buffer) - 1, -1, -1):
    company_recommendations[df['company_txt'][index_map[buffer[i]]]] = buffer[i]
    if len(company_recommendations) == 3:
        break
print(company_recommendations.values(), np.max(score_arr))

In [ ]:
student_previous_job_df = student_previous_job_df.replace(1.0, 0.0)
student_df = pd.DataFrame()
company_size, cost_of_living = [], []
for match in company_recommendations.values():
    company_size.append(df['Size'][index_map[match]])
    city = df['Location'][index_map[match]].split(',')[0]
    state = df['Location'][index_map[match]].split(',')[1]
    cost_of_living_val = cost_of_living_df[(cost_of_living_df['city'] == city) & (cost_of_living_df['state'] == state)]['cost_of_living'].values
    if len(cost_of_living_val) != 0:
        cost_of_living_val = cost_of_living_val[0]
    else:
        if state in cost_of_living_state:
            cost_of_living_val = cost_of_living_state[state]
        else:
            cost_of_living_val = cost_of_living_state['all']
    cost_of_living.append(cost_of_living_val)
    job_title = df['job_title_category'][index_map[match]]
    student_previous_job_df['col__' + job_title] = 1.0
company_size = [1 if 'Jan-50' in x else 1 if '51 - 200' in x
                                    else 2 if '201 - 500' in x else 3 if '501 - 1000' in x
                                    else 4 if '1001 - 5000' in x else 5 if '5001 - 10000' in x else 6 for x in company_size]
student_df['Size'] = company_size
student_df['cost_of_living'] = cost_of_living

temp = pd.concat([student_degree_df.iloc[: , 1:], student_previous_job_df.iloc[: , 1:], student_experience_df, student_skills_df.iloc[: , 1:]], axis=1)
temp = temp.append([temp] * (len(company_recommendations) - 1), ignore_index=True)
student_df = pd.concat([student_df, temp], axis=1)
student_df.head()

In [ ]:
skill_columns = list(student_skills_df.columns)
skill_columns.pop(0)
X_student = student_df[skill_columns]
W_student = tsvd.transform(X_student)
print(W.shape)
for i in range(W_student.shape[1]):
    col = 'skill_' + str(i)
    student_df[col] = W_student[:, i]
student_df = student_df.drop(skill_columns, axis=1)
student_df.head()

In [ ]:
salary_pred = results_xgb.predict(student_df)
print(salary_pred)

In [ ]:
recommendations = []
for idx, match in enumerate(company_recommendations.values()):
    components = {}
    components['company'] = df['company_txt'][index_map[match]]
    components['job_title'] = df['Job Title'][index_map[match]]
    components['location'] = df['Location'][index_map[match]]
    components['salary'] = salary_pred[idx]
    recommendations.append(components)
print(recommendations)

In [ ]:
# salary_df['Revenue'].value_counts()
# salary_df['Revenue'] = salary_df['Revenue'].apply(lambda x: 1 if 'Less than $1 million' in x else 1 if '$1 to $5 million' in x
#                                       else 1 if '$5 to $10 million' in x else 2 if '$10 to $25 million' in x
#                                       else 2 if '$25 to $50 million' in x else 2 if '$50 to $100 million' in x
#                                       else 3 if '$100 to $500 million' in x else 4 if '$500 million to $1 billion' in x
#                                       else 0 if 'Unknown' in x else 5)
# salary_df['Revenue'][salary_df['Revenue'] == 0] = salary_df['Size']
# salary_df['Revenue'][salary_df['Revenue'] > 5] = 5
# salary_df['Revenue'].value_counts()
# salary_df.head()

In [ ]:
# salary_df['Rating'] = salary_df['Rating'].apply(lambda x: 1 if x < 3 else 2 if x < 4 else 3)
# salary_df['Rating'].value_counts()
# salary_df['Age'] = salary_df['Age'].apply(lambda x: 1 if x < 5 else 2 if x < 10 else 3 if x < 15 else 4 if x < 20 else 5)
# salary_df['tech_hub'] = salary_df['tech_hub'].fillna(0)

In [ ]:
# skills_str = " ".join(skills_encode_df.columns)
# salary_df.to_csv('salary_input.csv', index=False)
# f = open('skills_encoded.txt', 'w')
# f.write(skills_str)
# f.close()
# salary_df = pd.read_csv('salary_input.csv')

In [ ]:
# X = salary_df.drop(['salary'], axis=1)
# Y = salary_df['salary']
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# print(X_train.shape, X_test.shape)

In [ ]:
# X_train_skill = X_train[skills_encode_df.columns]
# X_train_skill = X_train_skill.drop(['index'], axis=1)
# X_test_skill = X_test[skills_encode_df.columns]
# X_test_skill = X_test_skill.drop(['index'], axis=1)
# print(X_train_skill.shape)
# X_train_skill = X_train_skill.to_numpy()
# tsvd = TruncatedSVD(n_components=3, random_state=0)
# W = tsvd.fit_transform(X_train_skill)
# W_test = tsvd.transform(X_test_skill)
# print(W.shape)

In [ ]:
# for i in range(W.shape[1]):
#     col = 'skill_' + str(i)
#     X_train[col] = W[:, i]
#     X_test[col] = W_test[:, i]
# X_train = X_train.drop(skills_encode_df.columns, axis=1)
# X_test = X_test.drop(skills_encode_df.columns, axis=1)
# X_train.head()

In [ ]:
# train_df = pd.concat([X_train, y_train], axis=1)
# test_df = pd.concat([X_test, y_test], axis=1)
# train_df.to_csv('train.csv', index=False)
# test_df.to_csv('test.csv', index=False)

In [ ]:
# train_df = pd.read_csv('train.csv')
# test_df = pd.read_csv('test.csv')
# X_train = train_df.drop(['salary'], axis=1)
# y_train = train_df['salary']
# X_test = test_df.drop(['salary'], axis=1)
# y_test = test_df['salary']

In [ ]:
# X_train_LR = sm.add_constant(X_train)
# results_lr = sm.OLS(y_train, X_train_LR).fit()
# # print(results_lr.summary())
# X_test_LR = sm.add_constant(X_test)
# y_pred_LR = results_lr.predict(X_test_LR)
# mae = mean_absolute_error(y_test, y_pred_LR)
# print(mae)

In [ ]:
# results_rf = RandomForestRegressor(random_state=0)
# cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
# scores = cross_val_score(results_rf, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# scores = abs(scores)
# print('Mean MAE and Std from CV: %.3f (%.3f)' % (scores.mean(), scores.std()))
# results_rf = RandomForestRegressor(random_state=0).fit(X_train, y_train)
# y_pred_RF = results_rf.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred_RF)
# print('MAE from test:', mae)

In [ ]:
# importance = results_rf.feature_importances_
# for i,v in enumerate(importance):
#     print(f'Feature: {X_train.columns[i]}, Score: {v}')
# plt.bar([x for x in range(len(importance))], importance)

In [ ]:
# def xgb_regress(max_depth, n_estimators, learning_rate, subsample):
#     params_gbm = {}
#     params_gbm['max_depth'] = round(max_depth)
#     params_gbm['n_estimators'] = round(n_estimators)
#     params_gbm['learning_rate'] = learning_rate
#     params_gbm['subsample'] = subsample
#     scores = cross_val_score(GradientBoostingRegressor(random_state=0, **params_gbm),
#                              X, Y, scoring='neg_mean_absolute_error', cv=5)
#     return -np.mean(abs(scores))

# start = time.time()
# params_gbm ={
#     'max_depth':(6, 8),
#     'n_estimators':(400, 700),
#     'learning_rate':(0.05, 0.1),
#     'subsample':(0.7, 0.9)
# }
# gbm_bo = BayesianOptimization(xgb_regress, params_gbm, random_state=0)
# gbm_bo.maximize(init_points=20, n_iter=20)

In [ ]:
# results_xgb = XGBRegressor(n_estimators=600, learning_rate=0.1, subsample=0.7, max_depth=7)
# cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
# scores = cross_val_score(results_xgb, X_train_shrinked, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# scores = abs(scores)
# print('Mean MAE and Std from CV: %.3f (%.3f)' % (scores.mean(), scores.std()))
# results_xgb = XGBRegressor(random_state=0).fit(X_train_shrinked, y_train)
# y_pred_XGB = results_xgb.predict(X_test_shrinked)
# mae = mean_absolute_error(y_test, y_pred_XGB)
# print('MAE from test:', mae)

In [ ]:
# importance = results_xgb.feature_importances_
# for i,v in enumerate(importance):
#     print(f'Feature: {X_train_shrinked.columns[i]}, Score: {v}')
# plt.bar([x for x in range(len(importance))], importance)